In [1]:
import re
import io
import requests
import pandas as pd
from fuzzywuzzy import fuzz, process
from pac_hunter.states import us_state_to_abbrev, abbrev_to_us_state
from pac_hunter.utils import fuzzy_filter, clean_df
from pac_hunter.constants import OPENSECRETS_API_KEY, OPENFEC_API_KEY

c:\Users\Kepler\fraypac\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
raypac_house_2020 = pd.read_csv("raypac-house-2020.csv")
raypac_senate_2020 = pd.read_csv("raypac-senate-2020.csv")
jan6_house = pd.read_csv("jan6-house-deniers.csv")
jan6_senate = pd.read_csv("jan6-senate-deniers.csv")
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/election-deniers/fivethirtyeight_election_deniers.csv"
ballot_deniers = pd.read_csv(url)
ballot_deniers = ballot_deniers.assign(party="r")

In [3]:
raypac_2020 = pd.concat([raypac_house_2020.assign(office="Representative"), raypac_senate_2020.assign(office="Senator")], ignore_index=True)
jan6_congress = pd.concat([jan6_house.assign(office="Representative"), jan6_senate.assign(office="Senator")], ignore_index=True)

In [4]:
raypac_2020_clean = clean_df(raypac_2020)
jan6_congress_clean = clean_df(jan6_congress)
ballot_deniers_clean = clean_df(ballot_deniers)

In [5]:
jan6_deniers = jan6_congress_clean.query("vote=='yea'")

In [6]:
raypac_2020_deniers = fuzzy_filter(raypac_2020_clean, jan6_deniers, 'candidate', 'name')

In [7]:
raypac_2020_deniers

,candidate,party,state,total,cycle,office,total_float,name,state_code,matched_denier
1,devin nunes,r,calif,"$20,000",2020,representative,20000.0,candidate,ca,nunes
2,robert b aderholt,r,ala,"$10,000",2020,representative,10000.0,candidate,al,aderholt
4,richard w allen,r,ga,"$10,000",2020,representative,10000.0,candidate,ga,allen
5,brian babin,r,texas,"$10,000",2020,representative,10000.0,candidate,tx,babin
7,jim banks,r,ind,"$10,000",2020,representative,10000.0,candidate,in,banks
19,tom cole,r,okla,"$10,000",2020,representative,10000.0,candidate,ok,cole
34,michael guest,r,miss,"$10,000",2020,representative,10000.0,candidate,ms,guest
35,vicky hartzler,r,mo,"$10,000",2020,representative,10000.0,candidate,mo,hartzler
48,kevin mccarthy,r,calif,"$10,000",2020,representative,10000.0,candidate,ca,mccarthy
54,steven palazzo,r,miss,"$10,000",2020,representative,10000.0,candidate,ms,palazzo


In [8]:
total_donations = raypac_2020_deniers["total_float"].sum()
print(f"Raytheon Technologies via RAYPAC donated ${total_donations:.2f} to members of congress that voted against certifying the 2020 election in the 2019-2020 election cycle")

Raytheon Technologies via RAYPAC donated $235500.00 to members of congress that voted against certifying the 2020 election in the 2019-2020 election cycle


In [9]:
from pac_hunter.constants import OPENSECRETS_API_KEY


In [10]:
opensec_url = "https://www.opensecrets.org/api/"
org_name = "raytheon tech"
payload = dict(
    method="getOrgs",
    apikey = OPENSECRETS_API_KEY,
    org = org_name,
    output="json",
)
res = requests.get(opensec_url, params=payload)
org = res.json()["response"]["organization"]["@attributes"]

In [11]:
payload = dict(
    method="orgSummary",
    apikey=OPENSECRETS_API_KEY,
    id=org["orgid"],
    output="json",
)
res = requests.get(opensec_url, params=payload)

In [12]:
res.json()

{'response': {'organization': {'@attributes': {'cycle': '2022',
    'orgid': 'D000072615',
    'orgname': 'Raytheon Technologies',
    'total': '2168904',
    'indivs': '512633',
    'pacs': '1628500',
    'soft': '27771',
    'tot527': '0',
    'dems': '1021946',
    'repubs': '1102085',
    'lobbying': '5900000',
    'outside': '0',
    'mems_invested': '0',
    'gave_to_pac': '2521',
    'gave_to_party': '296598',
    'gave_to_527': '0',
    'gave_to_cand': '1431279',
    'source': 'www.opensecrets.org/orgs/summary.php?id=D000072615'}}}}

In [13]:
url_openfec = "https://api.open.fec.gov/v1"

In [14]:
path = "/names/committees/"
payload = dict(
    api_key = OPENFEC_API_KEY,
    q = "raytheon technologies"
)
res = requests.get(f"{url_openfec}{path}", params=payload)
committee_id = res.json()["results"][0]["id"]

In [49]:
payload = dict(
    api_key = OPENFEC_API_KEY,
    name = "devin nunes",
)
res = requests.get(f"{url_openfec}/candidates/search/", params=payload)
candidate_id = res.json()['results'][0]['candidate_id']

In [145]:
payload = dict(
    api_key = OPENFEC_API_KEY,
    committee_id = [committee_id],
    recipient_name = jan6_deniers['name'].to_numpy(),
    per_page = 100,
)
res = requests.get(f"{url_openfec}/schedules/schedule_b/by_recipient/", params=payload)

In [146]:
res.json()

{'api_version': '1.0',
 'pagination': {'pages': 5, 'per_page': 100, 'count': 456},
 'results': [{'committee_id': 'C00097568',
   'committee_total_disbursements': 3738189.07,
   'recipient_disbursement_percent': 0.27,
   'memo_total': 0.0,
   'recipient_name': 'COMMON GOOD VA',
   'cycle': 2014,
   'total': 10000.0,
   'memo_count': 0,
   'count': 2},
  {'committee_id': 'C00097568',
   'committee_total_disbursements': 3122751.44,
   'recipient_disbursement_percent': 0.32,
   'memo_total': 0.0,
   'recipient_name': 'FRIENDS OF BILL POSEY',
   'cycle': 2012,
   'total': 10000.0,
   'memo_count': 0,
   'count': 7},
  {'committee_id': 'C00097568',
   'committee_total_disbursements': 3722689.34,
   'recipient_disbursement_percent': 0.03,
   'memo_total': 0.0,
   'recipient_name': 'FRIENDS OF MATT GAETZ',
   'cycle': 2018,
   'total': 1000.0,
   'memo_count': 0,
   'count': 1},
  {'committee_id': 'C00097568',
   'committee_total_disbursements': 2730230.62,
   'recipient_disbursement_percent':